In [3]:
# native python libraries imports 
import math
from time import time
import os
from unittest import TestCase

# Data Handling
import numpy as np
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn imports 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# keras imports 
import tensorflow
import keras
from keras import Sequential
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Flatten, Dense, Dropout, ReLU
from keras.initializers import GlorotUniform
from keras.activations import relu, sigmoid, tanh
from keras.optimizers import Adam, SGD, Nadam
from keras.utils import get_file
from keras_tuner.tuners import RandomSearch, BayesianOptimization, Sklearn
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.datasets import fashion_mnist

# required for compatibility between sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

%matplotlib inline

In [4]:
def load_quickdraw10(file = './quickdraw10.npz'):
    '''
    Retrieves the quickdraw dataset and separates into training and testing classes
    
    Parameters:
    ------------
    file: str: location of quickdraw file on local machine
    
    Returns:
    --------
    X_train: numpy array list[list[int]]]: pixel values for all images that the model will be trained on
    X_test: numpy array list[list[int]]: pixel values for all images that the model will test predictions on
    y_train: numpy array list[int]: list of answer values for what number the picture represents; used in training
    y_test: numpy array list[int]: list of answer values for what the picture represents; used to determine model acc
    '''
    data = np.load(file)

    X = data['arr_0']
    Y = data['arr_1']
        
    X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_quickdraw10()

In [5]:
def create_model(n_layers, first_layer_nodes, last_layer_nodes, act_funct = "relu", negative_node_incrementation=True):
    
    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        assert n_layers >= 2, "n_layers must be 2 or greater"
        layers = []

        if negative_node_incrementation:
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
        else:
            nodes_increment = (first_layer_nodes - last_layer_nodes)/ (n_layers-1)

        nodes = first_layer_nodes

        for i in range(1, n_layers + 1):
            layers.append(math.ceil(nodes))
            nodes += nodes_increment

        return layers
    
    # Get List of number of nodes for each layer
    n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
    
    # Build Model
    model = Sequential()
    
    for i in range(1, n_layers):                  # Hidden Layers
        if i == 1:
            model.add(Dense(first_layer_nodes, input_dim = X_train.shape[1], activation = act_funct))
        else:
            model.add(Dense(n_nodes[i-1], activation = act_funct))
            
    model.add(Dense(10, activation='softmax'))    # Output Layer
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    return model


In [6]:
def simpler_model(hp):
    # instantiate model
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    model.compile(
        optimizer=keras.optimizers.Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

In [15]:
def created_model(hp):
    
    def gen_layer_nodes(hp):     # Function to automate number of nodes per layer
        
        layers = []
        first_layer_nodes = hp.get('units')    # Starting layer nodes: a hyperparameter
        last_layer_nodes = 100
        num_layers = hp.get('layers')          # Number of total layers: a hyperparameter
        
        nodes_increment = (last_layer_nodes - first_layer_nodes)/ (num_layers-1)
        nodes = first_layer_nodes

        for i in range(1, num_layers + 1):
            layers.append(math.ceil(nodes))
            nodes += nodes_increment

        return layers               # List of node count for each layer
     
    n_nodes = gen_layer_nodes(hp)   # Get List of node count for each layer by calling function
    num_layers = hp.get('layers')
    
    # Build Model
    model = Sequential() 
    for i in range(1, num_layers):                  
        if i == 1:                                # Input Layer
            model.add(Dense(hp.get('units'), input_dim = X_train.shape[1], activation = hp.get('activation')))
        else:                                     # Hidden Layers
            model.add(Dense(n_nodes[i-1], activation = hp.get('activation')))
            model.add(Dropout(hp.get('drop_prob')))      
    model.add(Dense(10, activation='softmax'))    # Output Layer
    
    # Compile model
    model.compile(loss = 'sparse_categorical_crossentropy', 
                  optimizer = keras.optimizers.Nadam(hp.get('learning_rate')), 
                  metrics = ['accuracy'])
    
    return model



In [17]:
# Pick hyperparameters
hp = HyperParameters()
hp.Int('units', min_value = 250, max_value = 750, step = 100)                    # Node Count
hp.Int('layers', min_value = 2, max_value = 6, step = 1)                        # Layer Count
hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3])                         # Learning Rate
hp.Choice('activation', values = ['relu', 'sigmoid', 'tanh'])                   # Activation
hp.Choice('drop_prob', values = [0.0, 0.2, 0.4])                                # Dropout

stop = EarlyStopping(monitor = 'val_loss', min_delta = .001, patience = 2)      # Early Stopping: Epochs

number_unique_parameter_combinations = (500 / 100) * (4 / 1) * 3 * 3 * 3
max_trials = number_unique_parameter_combinations * .1
max_trials

54.0

In [39]:
max_trials = num_unique_hp_combs * .05
num_initial_points = 10
beta = 3.0

# Bayesian search set up
bayesian_tuner = BayesianOptimization(
                                      created_model,
                                      objective = 'val_accuracy',
                                      max_trials = max_trials,
                                      hyperparameters = hp,
                                      num_initial_points = num_initial_points, 
                                      beta = beta, 
                                      seed = 1234,
                                      directory = './keras-tuner-trial',
                                      project_name = 'bayesian_optimization_5')

# Run the baysian search 

bayesian_tuner.search(X_train,
                      y_train,
                      epochs = 10,
                      validation_data = (X_test, y_test),
                      callbacks = [stop])

Trial 87 Complete [00h 00m 54s]
val_accuracy: 0.8461999893188477

Best val_accuracy So Far: 0.8586400151252747
Total elapsed time: 01h 18m 37s
INFO:tensorflow:Oracle triggered exit


In [43]:
# bayesian_tuner.results_summary()

print('Best Validation Score: 0.8586')
print()
print('Dynamic')
print()
print('Units: 600')
print('Layers: 8')
print('Learning Rate: 0.001')
print('Activation: Relu')
print('Dropout Rate: 0.0%')
print('Static')
print()
print('Optimizer: Adam')
print('Batch Size: 32')

Best Validation Score: 0.8586

Dynamic

Units: 600
Layers: 8
Learning Rate: 0.001
Activation: Relu
Dropout Rate: 0.0%
Static

Optimizer: Adam
Batch Size: 32


In [10]:
num_initial_points = 8
beta = 3.0

# Bayesian search set up
bayesian_tuner = BayesianOptimization(
                                      created_model,
                                      objective = 'val_accuracy',
                                      max_trials = max_trials,
                                      hyperparameters = hp,
                                      num_initial_points = num_initial_points, 
                                      beta = beta, 
                                      seed = 1234,
                                      directory = './keras-tuner-trial3',
                                      project_name = 'bayesian_optimization_5')

# Run the baysian search 
# batch_sizes = [64, 128, 256, 512]
# for batch_size in batch_sizes:
bayesian_tuner.search(X_train,
                      y_train,
                      epochs = 10,
                      validation_data = (X_test, y_test),
                      batch_size = 32,
                      callbacks = [stop])

Trial 33 Complete [00h 02m 57s]
val_accuracy: 0.8725600242614746

Best val_accuracy So Far: 0.8763200044631958
Total elapsed time: 01h 23m 58s
INFO:tensorflow:Oracle triggered exit


In [12]:
bayesian_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial3/bayesian_optimization_5
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 750
layers: 6
learning_rate: 0.001
activation: relu
drop_prob: 0.0
Score: 0.8763200044631958
Trial summary
Hyperparameters:
units: 675
layers: 6
learning_rate: 0.001
activation: relu
drop_prob: 0.0
Score: 0.8729599714279175
Trial summary
Hyperparameters:
units: 675
layers: 6
learning_rate: 0.001
activation: relu
drop_prob: 0.0
Score: 0.8727200031280518
Trial summary
Hyperparameters:
units: 300
layers: 8
learning_rate: 0.001
activation: relu
drop_prob: 0.0
Score: 0.8725600242614746
Trial summary
Hyperparameters:
units: 525
layers: 8
learning_rate: 0.001
activation: relu
drop_prob: 0.0
Score: 0.8725600242614746
Trial summary
Hyperparameters:
units: 675
layers: 6
learning_rate: 0.001
activation: relu
drop_prob: 0.0
Score: 0.8725600242614746
Trial summary
Hyperparameters:
units: 675
layers: 6
learning_rate: 0.00